## Interaktiivinen Phi 3 Mini 4K Instruct Chatbot Whisperillä

### Johdanto:
Interaktiivinen Phi 3 Mini 4K Instruct Chatbot on työkalu, jonka avulla käyttäjät voivat olla vuorovaikutuksessa Microsoft Phi 3 Mini 4K instruct -demon kanssa tekstin tai äänen avulla. Chatbotia voidaan käyttää monenlaisiin tehtäviin, kuten käännöksiin, säätietoihin ja yleiseen tiedonhankintaan.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Luo uusi token  
Anna uusi nimi  
Valitse kirjoitusoikeudet  
Kopioi token ja tallenna se turvalliseen paikkaan


Seuraava Python-koodi suorittaa kaksi päätehtävää: `os`-moduulin tuomisen ja ympäristömuuttujan asettamisen.

1. `os`-moduulin tuominen:
   - Pythonin `os`-moduuli tarjoaa tavan olla vuorovaikutuksessa käyttöjärjestelmän kanssa. Sen avulla voi suorittaa erilaisia käyttöjärjestelmään liittyviä tehtäviä, kuten käyttää ympäristömuuttujia, käsitellä tiedostoja ja hakemistoja jne.
   - Tässä koodissa `os`-moduuli tuodaan käyttöön `import`-lauseella. Tämä lause tekee `os`-moduulin toiminnallisuudet käytettäväksi nykyisessä Python-skriptissä.

2. Ympäristömuuttujan asettaminen:
   - Ympäristömuuttuja on arvo, johon käyttöjärjestelmän ohjelmat voivat päästä käsiksi. Se on tapa tallentaa asetuksia tai muuta tietoa, jota useat ohjelmat voivat käyttää.
   - Tässä koodissa uusi ympäristömuuttuja asetetaan käyttämällä `os.environ`-sanakirjaa. Sanakirjan avain on `'HF_TOKEN'`, ja arvo määritetään `HUGGINGFACE_TOKEN`-muuttujasta.
   - `HUGGINGFACE_TOKEN`-muuttuja määritellään juuri tämän koodinpätkän yläpuolella, ja sille annetaan merkkijonoarvo `"hf_**************"` käyttäen `#@param`-syntaksia. Tätä syntaksia käytetään usein Jupyter-notebookeissa, jotta käyttäjä voi syöttää arvoja ja määrittää parametreja suoraan notebookin käyttöliittymässä.
   - Asettamalla `'HF_TOKEN'`-ympäristömuuttujan, sitä voivat käyttää ohjelman muut osat tai muut ohjelmat, jotka toimivat samalla käyttöjärjestelmällä.

Kaiken kaikkiaan tämä koodi tuo `os`-moduulin käyttöön ja asettaa ympäristömuuttujan nimeltä `'HF_TOKEN'` arvolla, joka on määritetty `HUGGINGFACE_TOKEN`-muuttujassa.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Tämä koodinpätkä määrittää clear_output-nimisen funktion, jota käytetään nykyisen solun tulosteen tyhjentämiseen Jupyter Notebookissa tai IPythonissa. Tarkastellaan koodia ja sen toiminnallisuutta tarkemmin:

Funktio clear_output ottaa yhden parametrin nimeltä wait, joka on totuusarvo (boolean). Oletuksena wait on asetettu arvoon False. Tämä parametri määrittää, odottaako funktio uuden tulosteen ilmestymistä ennen olemassa olevan tulosteen tyhjentämistä.

Itse funktiota käytetään nykyisen solun tulosteen tyhjentämiseen. Jupyter Notebookissa tai IPythonissa, kun solu tuottaa tulostetta, kuten tulostettua tekstiä tai graafisia kuvioita, tämä tuloste näytetään solun alapuolella. Funktio clear_output mahdollistaa kyseisen tulosteen tyhjentämisen.

Funktion toteutusta ei ole annettu koodinpätkässä, kuten ellipsistä (...) voi päätellä. Ellipsi toimii paikkamerkkinä varsinaiselle koodille, joka suorittaa tulosteen tyhjentämisen. Funktion toteutus saattaa sisältää vuorovaikutusta Jupyter Notebookin tai IPythonin API:n kanssa, jotta olemassa oleva tuloste voidaan poistaa solusta.

Kaiken kaikkiaan tämä funktio tarjoaa kätevän tavan tyhjentää nykyisen solun tuloste Jupyter Notebookissa tai IPythonissa, mikä helpottaa tulosteen hallintaa ja päivittämistä interaktiivisten koodausistuntojen aikana.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Suorita tekstistä puheeksi (TTS) Edge TTS -palvelun avulla. Käydään läpi olennaiset funktioiden toteutukset yksi kerrallaan:

1. `calculate_rate_string(input_value)`: Tämä funktio ottaa syötteenä arvon ja laskee TTS-äänen nopeusmerkkijonon. Syöte edustaa puheen haluttua nopeutta, jossa arvo 1 tarkoittaa normaalia nopeutta. Funktio laskee nopeusmerkkijonon vähentämällä 1 syötteestä, kertomalla sen 100:lla ja määrittämällä etumerkin sen perusteella, onko syöte suurempi tai yhtä suuri kuin 1. Funktio palauttaa nopeusmerkkijonon muodossa "{sign}{rate}".

2. `make_chunks(input_text, language)`: Tämä funktio ottaa syötteenä tekstin ja kielen. Se jakaa syötteen tekstin osiin kieleen liittyvien sääntöjen perusteella. Tässä toteutuksessa, jos kieli on "English", funktio jakaa tekstin pisteen (".") kohdalta ja poistaa mahdolliset alussa tai lopussa olevat välilyönnit. Se lisää pisteen jokaisen osan loppuun ja palauttaa suodatetun listan osista.

3. `tts_file_name(text)`: Tämä funktio luo tiedostonimen TTS-äänitiedostolle syötteenä annetun tekstin perusteella. Se tekee useita muunnoksia tekstille: poistaa lopussa olevan pisteen (jos sellainen on), muuntaa tekstin pieniksi kirjaimiksi, poistaa alussa ja lopussa olevat välilyönnit ja korvaa välilyönnit alaviivoilla. Se lyhentää tekstin enintään 25 merkkiin (jos pidempi) tai käyttää koko tekstiä, jos se on tyhjä. Lopuksi se luo satunnaisen merkkijonon [`uuid`] -moduulin avulla ja yhdistää sen lyhennettyyn tekstiin luodakseen tiedostonimen muodossa "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Tämä funktio yhdistää useita äänitiedostoja yhdeksi tiedostoksi. Se ottaa syötteenä listan äänitiedostojen polkuja ja ulostulopolun. Funktio alustaa tyhjän `AudioSegment`-objektin nimeltä [`merged_audio`]. Se käy läpi jokaisen äänitiedoston polun, lataa äänitiedoston `AudioSegment.from_file()` -metodilla `pydub`-kirjastosta ja lisää nykyisen äänitiedoston [`merged_audio`]-objektiin. Lopuksi se vie yhdistetyn äänen määritettyyn ulostulopolkuun MP3-muodossa.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Tämä funktio suorittaa TTS-toiminnon Edge TTS -palvelun avulla. Se ottaa syötteenä tekstiosien listan, puheen nopeuden, äänen nimen ja tallennuspolun. Jos osien määrä on suurempi kuin 1, funktio luo hakemiston yksittäisten osien äänitiedostojen tallentamista varten. Se käy läpi jokaisen osan, rakentaa Edge TTS -komennon `calculate_rate_string()` -funktion, äänen nimen ja osatekstin avulla ja suorittaa komennon `os.system()` -funktiolla. Jos komennon suoritus onnistuu, se lisää luodun äänitiedoston polun listaan. Kun kaikki osat on käsitelty, se yhdistää yksittäiset äänitiedostot `merge_audio_files()` -funktion avulla ja tallentaa yhdistetyn äänen määritettyyn tallennuspolkuun. Jos osia on vain yksi, se luo suoraan Edge TTS -komennon ja tallentaa äänen tallennuspolkuun. Lopuksi se palauttaa luodun äänitiedoston tallennuspolun.

6. `random_audio_name_generate()`: Tämä funktio luo satunnaisen äänitiedoston nimen [`uuid`] -moduulin avulla. Se luo satunnaisen UUID:n, muuntaa sen merkkijonoksi, ottaa ensimmäiset 8 merkkiä, lisää ".mp3"-päätteen ja palauttaa satunnaisen äänitiedoston nimen.

7. `talk(input_text)`: Tämä funktio on pääasiallinen sisääntulopiste TTS-toiminnon suorittamiseksi. Se ottaa syötteenä tekstin. Se tarkistaa ensin tekstin pituuden määrittääkseen, onko kyseessä pitkä lause (vähintään 600 merkkiä). Pituuden ja `translate_text_flag`-muuttujan arvon perusteella se määrittää kielen ja luo tekstiosien listan `make_chunks()` -funktion avulla. Se luo sitten tallennuspolun äänitiedostolle `random_audio_name_generate()` -funktion avulla. Lopuksi se kutsuu `edge_free_tts()` -funktion suorittamaan TTS-toiminnon ja palauttaa luodun äänitiedoston tallennuspolun.

Yhteenvetona nämä funktiot toimivat yhdessä jakaakseen syötteen tekstin osiin, luodakseen tiedostonimen äänitiedostolle, suorittaakseen TTS-toiminnon Edge TTS -palvelun avulla ja yhdistääkseen yksittäiset äänitiedostot yhdeksi tiedostoksi.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Kahden funktion, convert_to_text ja run_text_prompt, toteutus sekä kahden luokan, str ja Audio, määrittely.

convert_to_text-funktio ottaa syötteenä audio_path-parametrin ja muuntaa äänen tekstiksi käyttämällä mallia nimeltä whisper_model. Funktio tarkistaa ensin, onko gpu-lippu asetettu arvoon True. Jos se on, whisper_modelia käytetään tietyillä parametreilla, kuten word_timestamps=True, fp16=True, language='English' ja task='translate'. Jos gpu-lippu on False, whisper_modelia käytetään parametrilla fp16=False. Tuloksena saatu transkriptio tallennetaan tiedostoon nimeltä 'scan.txt' ja palautetaan tekstinä.

run_text_prompt-funktio ottaa syötteenä viestin ja chat_history-parametrin. Se käyttää phi_demo-funktiota luodakseen chatbotin vastauksen syötteenä annetun viestin perusteella. Generoitu vastaus välitetään talk-funktiolle, joka muuntaa vastauksen äänenä tiedostoksi ja palauttaa tiedostopolun. Audio-luokkaa käytetään äänen näyttämiseen ja toistamiseen. Ääni näytetään IPython.display-moduulin display-funktion avulla, ja Audio-objekti luodaan autoplay=True-parametrilla, jolloin ääni alkaa toistua automaattisesti. chat_history päivitetään syötteenä annetulla viestillä ja generoidulla vastauksella, ja palautetaan tyhjä merkkijono sekä päivitetty chat_history.

str-luokka on Pythonin sisäänrakennettu luokka, joka edustaa merkkijonojen sekvenssiä. Se tarjoaa erilaisia metodeja merkkijonojen käsittelyyn ja manipulointiin, kuten capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill ja muita. Näiden metodien avulla voi suorittaa operaatioita, kuten etsimistä, korvaamista, muotoilua ja merkkijonojen manipulointia.

Audio-luokka on mukautettu luokka, joka edustaa ääniobjektia. Sitä käytetään luomaan äänisoitin Jupyter Notebook -ympäristössä. Luokka hyväksyy useita parametreja, kuten data, filename, url, embed, rate, autoplay ja normalize. data-parametri voi olla numpy-taulukko, näytteiden lista, merkkijono, joka edustaa tiedostonimeä tai URL-osoitetta, tai raakaa PCM-dataa. filename-parametria käytetään määrittämään paikallinen tiedosto, josta ääni data ladataan, ja url-parametria käytetään määrittämään URL-osoite, josta ääni data ladataan. embed-parametri määrittää, tulisiko ääni data upottaa data-URI:n avulla vai viitata alkuperäiseen lähteeseen. rate-parametri määrittää ääni datan näytteenottotaajuuden. autoplay-parametri määrittää, alkaako ääni toistua automaattisesti. normalize-parametri määrittää, tulisiko ääni data normalisoida (skaalata) maksimaaliseen mahdolliseen alueeseen. Audio-luokka tarjoaa myös metodeja, kuten reload, ääni datan lataamiseen uudelleen tiedostosta tai URL-osoitteesta, sekä attribuutteja, kuten src_attr, autoplay_attr ja element_id_attr, HTML-äänielementin vastaavien attribuuttien hakemiseen.

Yhteenvetona nämä funktiot ja luokat käytetään äänen transkriptioon tekstiksi, chatbotin ääni vastausten luomiseen sekä äänen näyttämiseen ja toistamiseen Jupyter Notebook -ympäristössä.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
